In [25]:
import pandas as pd # pandas kütüphanesini "pd" takma adıyla aktarıyor

# Şimdi de verileri çağırma vakti! CSV dosyasını alıp elimize alıyoruz.
veri = pd.read_csv("malicious_phish.csv")

# İşte burada csv dosyasının ilk 10 satırını gösteriyorum.
veri.head()

,url,type
0,br-icloud.com.br,phishing
1,mp3raid.com/music/krizz_kaliko.html,benign
2,bopsecrets.org/rexroth/cr/1.htm,benign
3,http://www.garage-pirenne.be/index.php?option=...,defacement
4,http://adventure-nicaragua.net/index.php?optio...,defacement


In [26]:
# Burada CSV dosyasının özet bilgilerini alıyoruz. Hangi sütunlar var, kaç tane değer dolu, kaçı eksik, türleri neler gibi.
veri.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 651191 entries, 0 to 651190
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   url     651191 non-null  object
 1   type    651191 non-null  object
dtypes: object(2)
memory usage: 9.9+ MB


In [27]:
veri = pd.read_csv("malicious_phish.csv")
veri['type'].value_counts() # 'type' adlı sütündaki değerlerin sayısını sayıp tekrarlanma durumunu verir

benign        428103
defacement     96457
phishing       94111
malware        32520
Name: type, dtype: int64

In [28]:
# CSV dosyasındaki verileri çekip elimize alıyoruz.
veri = pd.read_csv("malicious_phish.csv")
# 'url' adlı sütunda hangi değerden kaçar tane olduğunu sayarak tekrarlanma durumunu anlayabiliriz.
# Böylece en popüler URL'leri ve nadir bulunanları görebiliriz.
veri['url'].value_counts()
# İşte burada sayılarına bakarak hangi URL'lerin daha fazla tekrarlandığını keşfedebiliriz.

http://style.org.hc360.com/css/detail/mysite/siteconfig/pro_control.css                         180
http://www.pubyun.com/wiki/lib/exe/css.php?t=dokuwiki&tseed=f1f16e8c6142fab8553f2b9cfe045961    146
http://9779.info/%E6%A0%91%E5%8F%B6%E8%B4%B4%E7%94%BB%E4%BD%9C%E5%93%81/                         91
http://9779.info/%E6%A0%91%E5%8F%B6%E7%B2%98%E8%B4%B4%E7%94%BB/                                  88
http://www.js182.com/app/member/SportOption.php?uid=guest&langx=gb                               79
                                                                                               ... 
en.wikipedia.org/wiki/Montrealer                                                                  1
redbulgaria.com/                                                                                  1
news.ontario.ca/opo/en/2011/06/highway-417-expansion-moving-forward.html                          1
metal-archives.com/bands/Goddess_of_Desire/492                                                    1


In [30]:
# Şimdi, bir şekilde verilerimizi okuyoruz. CSV dosyasındaki verilerimizi elimize alalım
veri = pd.read_csv("malicious_phish.csv")

# Biraz toparlanma zamanı. Aynı URL'ye sahip tekrarlayan satırları yakalayıp birinci sürümünü tutuyoruz.
# Bu sayede veri setimiz daha temiz ve düzenli hale geliyor.
veri = veri.drop_duplicates(subset='url', keep='first')

# Her sütunda kaç tane eksik değer olduğunu sayarak verimizi inceliyoruz.
# Belki de eksik değerleri dönüştürme yaparız.
veri.isnull().sum()

# bu kod satırı bize eksik değerlerin toplam sayısını getiriyor.
# Eğer her şey yolunda gitmezse, eksik değerleri bir şekilde tamamlamaya başvurabiliriz!

url     0
type    0
dtype: int64

In [31]:
# Burada sklearn kütüphanesinin train_test_split ve TfidfVectorizer araçlarını kullanarak veriyi eğitim ve test kümelerine ayırıyorum.
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

# X değişkenine URL'leri, y değişkenine ise tür etiketlerini atıyorum.
X = veri['url']
y = veri['type']

# Şimdi verileri ayırmak zamanı. %20'sini test için saklayacağım, random_state sayesinde her seferinde aynı sonuçları alacağım.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# TF-IDF nedir dersen, kelime önemini ölçmekte kullanılan bir araç. Sanki benim için önemli notları bulup vurguluyor gibi düşün.
# TF-IDF vektörleştiriciyi hazırlıyorum, daha sonra verileri ona sokup dönüşümler yapacağım.
tfidf_vectorizer = TfidfVectorizer()

# Eğitim verilerini TF-IDF vektörlerine dönüştürüyorum. Yani kelimeleri sayılara çeviriyorum,ama önem sıralamasını da atlamıyorum tabii ki!
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)

# Şimdi ise test verilerini TF-IDF vektörlerine dönüştürüyorum. Yani elimdeki öğrenilen kelime sıralamasını kullanarak test verilerini de sayılara çeviriyorum.
X_test_tfidf = tfidf_vectorizer.transform(X_test)


In [34]:
# Burada sklearn kütüphanesinden LogisticRegression ve accuracy_score araçlarını kullanarak modeli oluşturuyorum ve değerlendiriyorum.
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Önce modelimi oluşturuyorum.
model = LogisticRegression(max_iter=1000)
# Şimdi de modelimi eğitim verileriyle eğitiyorum. Bakalım bu model verileri nasıl öğrenecek?
model.fit(X_train_tfidf, y_train)
# Eğitim bitti, sıra test verileriyle modelimi sınamakta.
# Modelim elimdeki TF-IDF vektörleriyle test verilerini tahmin edecek.
y_pred = model.predict(X_test_tfidf)
# Tahminlerimizi gerçek etiketlerle karşılaştırarak ne kadar başarılı olduğumuza bakalım.
# Bu yüzden accuracy_score kullanıyorum. Ne kadar başarılı göreceğiz.
accuracy = accuracy_score(y_test, y_pred)
# Sonuçları ekrana yazdırmak istiyorum. Umarım yüksek bir doğruluk payı çıkar!
print("Model Başarı Oranı:", accuracy)

Model Başarı Oranı: 0.946429685550287
